In [9]:
import pandas as pd
import numpy as np

In [ ]:
path_test_clean = '../data/cleaned_data/test_correct.csv'
test_file= pd.read_csv(path_test_clean)
lnr = test_file['LNR']
test_file.drop(['LNR'], axis=1, inplace=True)

In [2]:
from process_and_ml.pipeline import cat_features_fill_na

test_df_filled = cat_features_fill_na(test_file, cat_features=['CAMEO_DEU_2015', 'OST_WEST_KZ'])

In [3]:
from process_and_ml.pipeline import load_catboost_model

catboost = load_catboost_model('catboost_first.cbm')

In [4]:
#load file with D19 Sociales
from process_and_ml.data_wrangler import load_test_file

test, lnr = load_test_file('../data/train_test_files/Udacity_MAILOUT_052018_TEST.csv')

/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
d19_df = test['D19_SOZIALES']



In [ ]:
y_pred=catboost.predict_proba(test_df_filled)[:, 1]

In [7]:
df_19 = pd.DataFrame(d19_df)
df_19['pred'] = y_pred

In [8]:
df_19 = pd.DataFrame(d19_df)
df_19['pred'] = y_pred

In [1]:
df_19['pred_soc'] = np.where(df_19['D19_SOZIALES']==1, np.where(df_19['pred'] < 0.8, 1, df_19['pred']), df_19['pred'])


NameError: name 'np' is not defined

In [26]:
from helpers.kaggle_submission import kaggle_submission
kaggle_submission(
    column_lnr=lnr,
    y_pred=df_19['pred_soc'].values,
    submission_filename='catboost_third_shoot',
    submission_message='CatBoost trained with aumgument update = 1.')

100%|██████████| 896k/896k [00:04<00:00, 191kB/s]  


ApiException: (403)
Reason: Your team has used its submission allowance (5 of 5). This resets at midnight UTC (14 hours from now).
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 09 Mar 2021 09:18:55 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'ka_sessionid=653903763387701f949bd4dbee2cc0cc; max-age=2626560; path=/, GCLB=CPrp_YfJ27jQHw; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'Access-Control-Allow-Credentials': 'true', 'Turbolinks-Location': 'https://www.kaggle.com/api/v1/competitions/submissions/submit/udacity-arvato-identify-customers', 'X-Kaggle-MillisecondsElapsed': '43', 'X-Kaggle-RequestId': 'ed3e6e8d6337b3f962a87dacd15fd17e', 'X-Kaggle-ApiVersion': '1.5.10', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-7zHPwamQJCuKiNZn9H9GLA==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: h